In [16]:
import regex as re
import pandas as pd
import os
import random
import numpy as np

In [17]:
data = pd.read_csv('smalldata_washed.csv')
rumor = data['rumor'].to_list()
reverse = data['reverse'].to_list()
rumor_class = len(rumor)*[1]
reverse_class = len(reverse)*[0]
data = rumor + reverse
data_class = rumor_class + reverse_class
#后续添加数据要放在raw目录下，使用.xlsx格式将后续标注的数据加入data
raw_data_list = os.listdir('raw')
data_list = []
good_name = re.compile(r'^(?!(\~\$)).*(.xlsx)')
for i in raw_data_list:
    if good_name.match(i):
    #    data_list.append(i)
        temp = pd.read_excel('raw/'+i)
        temp.fillna('',inplace=True)
        temp_rumor = [x.strip() for x in  temp['punc_rumor'].to_list()  if x.strip()!='']
        temp_reverse = [x.strip() for x in  temp['punc_truth'].to_list()  if x.strip()!='']
        temp_rumor_class = len(temp_rumor)*[1]
        temp_reverse_class = len(temp_reverse)*[0]
        temp_data = temp_rumor + temp_reverse
        temp_data_class = temp_rumor_class + temp_reverse_class
        for i in range( len(temp_data)):
            temp_data[i]=re.sub(r'[\,\'\ ]|(\n)','',temp_data[i])
    data = data + temp_data
    data_class = data_class + temp_data_class
with open('data.txt','w') as f:
    for i in range(len(data)):
        f.write(data[i]+'\t'+str(data_class[i])+'\n')
        data_list =list( zip(data, data_class))
random.shuffle(data_list)
data, data_class = zip(*data_list)
index = ['rumor{}{}'.format(i, '!' * j) for (i, j) in zip(range(len(data)), data_class)]
df = pd .DataFrame(data=zip(data_class,data), columns=["rumor","text"], index=index)


In [18]:

# jieba分词
import jieba
# 去除停用词
with open ('hit_stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.read().splitlines()
 
seg_list = []
for i in range(len(df)):
    words = jieba.cut(df["text"].iloc[i],cut_all=False)
    words = [word for word in words if word not in stopwords]
    seg_list.append( ' '.join(words))
 
df['seg_text'] = seg_list
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.casual import casual_tokenize
tfidf_model = TfidfVectorizer(tokenizer=casual_tokenize,ngram_range=(1,2))
tfidf_docs = tfidf_model.fit_transform(raw_documents=df.seg_text).toarray()#这句话计算了tfidf
tfidf_docs.shape
from sklearn.metrics.pairwise import cosine_similarity
cos_matrix = cosine_similarity(tfidf_docs)
df['cos_matrix'] = cos_matrix.tolist()
#snowNLP情感分析
from snownlp import SnowNLP
sentimentslist = []
for i in (df['text']):
    a1 = SnowNLP(i)
    a2 = a1.sentiments
    sentimentslist.append(a2)
df['length'] = [len(x) for x in df['text']]
df['sentiments'] = sentimentslist

/Users/stevenyu/opt/anaconda3/envs/nlpiaenv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [19]:
from sklearn.decomposition import PCA
k=min(len(tfidf_model.vocabulary_),len(df))
k=100
pca = PCA(n_components=k)
pca_docs = pca.fit_transform(tfidf_docs)
base = sum(pca.singular_values_)
nums = pca.singular_values_.tolist()
pca = PCA(n_components=k)
pca_docs = pca.fit_transform(tfidf_docs)
pca_topic_vectors = pca.transform(tfidf_docs)
columns = ['topic{}'.format(i) for i in range(pca_docs.shape[1])]
pca_topic_vectors = pd.DataFrame(pca_topic_vectors, columns=columns, index=index)
pca_topic_vectors.round(3).head(6)



,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,...,topic90,topic91,topic92,topic93,topic94,topic95,topic96,topic97,topic98,topic99
rumor0,-0.006,0.042,0.016,0.001,-0.030,0.007,-0.024,-0.014,0.010,0.001,...,-0.005,-0.009,-0.040,0.015,0.027,-0.022,-0.014,0.001,0.008,-0.041
rumor1,0.000,-0.080,-0.005,0.004,-0.083,0.028,-0.011,0.019,-0.004,-0.063,...,-0.167,0.068,-0.096,0.056,-0.056,0.024,-0.040,-0.027,-0.144,0.085
rumor2,-0.020,0.003,-0.015,-0.038,0.005,0.015,-0.002,-0.010,-0.004,0.034,...,-0.044,0.003,0.031,0.006,-0.026,0.027,0.026,-0.013,-0.012,-0.012
rumor3,-0.021,0.033,0.020,-0.045,-0.028,0.020,-0.043,-0.040,-0.008,0.031,...,0.129,0.001,-0.037,-0.008,0.055,-0.030,-0.030,-0.017,0.075,-0.008
rumor4,-0.020,0.020,-0.034,-0.013,-0.015,0.008,-0.007,0.006,-0.007,0.020,...,-0.002,-0.008,-0.010,-0.005,0.004,-0.007,0.002,0.017,-0.002,-0.006
rumor5!,-0.017,0.024,-0.031,-0.023,0.002,0.012,-0.010,-0.023,-0.006,0.025,...,0.002,-0.008,-0.011,0.002,0.007,-0.007,0.003,-0.003,-0.004,-0.011


In [20]:
df['pca_topic_vectors'] = pca_topic_vectors.values.tolist()

In [21]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import casual_tokenize
np.random.seed(42)

counter = CountVectorizer(tokenizer=casual_tokenize)
bow_docs = pd.DataFrame(counter.fit_transform(raw_documents=df.text)
                        .toarray(), index=index)
column_nums, terms = zip(*sorted(zip(counter.vocabulary_.values(),
                                     counter.vocabulary_.keys())))
bow_docs.columns = terms
from sklearn.decomposition import LatentDirichletAllocation as LDiA
ldia = LDiA(n_components=32, learning_method='batch')
ldia = ldia.fit(bow_docs)  # <1>
ldia.components_.shape
ldia32_topic_vectors = ldia.transform(bow_docs)
columns32 = ['topic{}'.format(i) for i in range(ldia.components_.shape[0])]
ldia32_topic_vectors = pd.DataFrame(ldia32_topic_vectors,index=index, columns=columns32)
ldia32_topic_vectors.round(2).head()

/Users/stevenyu/opt/anaconda3/envs/nlpiaenv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,...,topic22,topic23,topic24,topic25,topic26,topic27,topic28,topic29,topic30,topic31
rumor0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
rumor1,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,...,0.02,0.02,0.02,0.02,0.02,0.02,0.52,0.02,0.02,0.02
rumor2,0.01,0.01,0.84,0.01,0.01,0.01,0.01,0.01,0.01,0.01,...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01
rumor3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
rumor4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.21,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [22]:
df['ldia_topic_vectors'] = ldia32_topic_vectors.values.tolist()

In [23]:
df.to_csv('../dataset/data.csv',encoding='utf-8')
data = pd.read_csv('../dataset/data.csv',encoding='utf-8',index_col=0)